# In this notebook we would train the catboost model

In [ ]:
drop_catboost_data_driven = [
    # Raw OHLC (massively correlated with close)
    'open', 'high', 'low',
    
    # ATR components
    'high_low', 'high_close', 'low_close',
    
    # Helper calculations  
    'typical_price',  # 99.99% corr with close
    'true_range',     # Captured in atr_14
    'volume_mean_20', # Used in ratio
    
    # Highly correlated MAs (keep EMA_7, drop others)
    'EMA_21',         # 99.99% corr with SMA_20
    'SMA_20',         # 99.99% corr with vwap_24h  
    'vwap_24h',       # Redundant with SMA_20
    'close_4h',       # 99.99% corr with close
    
    # Bollinger components (keep bb_position, width)
    'bollinger_upper', 'bollinger_lower',  # 99.98% corr with SMA_20
    
    # MACD components (keep histogram)
    'MACD_line', 'MACD_signal',  # 95% corr, histogram captures key info
    
    # Support/resistance (99.85% corr with each other)
    'resistance_level', 'support_level',  # Highly correlated
    
    # Zero-importance binary flags
    'vol_spike_1_5x',           # 0 LightGBM splits
    'near_upper_band', 'near_lower_band',  # 0 LightGBM splits
    'break_upper_band', 'break_lower_band', # 0 LightGBM splits
    'rsi_oversold',             # 0 LightGBM splits
    'above_sma20',              # 0 LightGBM splits, derivable
    'macd_positive',            # 0 LightGBM splits
    'volume_breakout', 'volume_breakdown',  # 0 LightGBM splits
    
    # Highly correlated position flags
    'above_sma50', 'ema7_above_ema21',  # 77% corr, derivable
    
    # Low-importance cross signals (keep best ones)
    'ema_cross_down',           # Lower importance than ema_cross_up
    
    # Some oscillator extremes (keep the reversals)
    'rsi_overbought', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold',
    
    # Trend flags vs continuous values
    'trending_market',          # Keep ADX value instead
    
    # All scenario features (very low importance)
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
]

In [ ]:
# =============================================================
#  Stable CatBoost F0.5 Optimizer (Windows‑friendly, CPU default)
#  – avoids Bayesian‑bootstrap "subsample" crash
#  – safe pos_weight bounds when classes ~balanced
# =============================================================

import os, gc, time, random, warnings, joblib, platform
from pathlib import Path

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)
from sklearn.model_selection import TimeSeriesSplit
import optuna

# ─── GLOBAL CONFIG ────────────────────────────────────────────
CSV_PATH     = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
VAL_FRAC     = 0.20              # fraction held out for final test
N_TRIALS     = 100               # hyper‑parameter trials
EARLY_STOP   = 150               # CatBoost early stopping
CV_FOLDS     = 3                 # chronological folds
FIXED_THRESH = 0.50              # threshold used inside CV
BETA         = 0.5               # F‑beta focus (precision‑heavy)
SEED         = 42

# ─── REPRODUCIBILITY ─────────────────────────────────────────
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
random.seed(SEED)
warnings.filterwarnings("ignore")

print(f"🖥️ Platform: {platform.system()} — running in CPU‑safe mode")

# ─── HELPERS ─────────────────────────────────────────────────

def fbeta_score_fixed(y_true, y_pred, beta: float = BETA, zero_division: float = 0.0):
    """Compute F‑beta where precision is weighted β<1 heavier (β=0.5)."""
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    if p == 0 and r == 0:
        return zero_division
    beta2 = beta ** 2
    return (1 + beta2) * p * r / (beta2 * p + r + 1e-9)

def cleanup_memory():
    gc.collect()

# ─── LOAD & PREPARE DATA ─────────────────────────────────────
print("📂 Loading data …")

df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df[df.index >= "2018-01-01"]
if "target" not in df.columns:
    raise ValueError("Column 'target' not found in dataset")

# feature leakage prevention — drop highly correlated / helper cols
DROP_COLS = [
    "open", "high", "low", "high_low", "high_close", "low_close",
    "typical_price", "true_range", "volume_mean_20", "EMA_21", "SMA_20",
    "vwap_24h", "close_4h", "bollinger_upper", "bollinger_lower",
    "MACD_line", "MACD_signal", "resistance_level", "support_level",
    "vol_spike_1_5x", "near_upper_band", "near_lower_band",
    "break_upper_band", "break_lower_band", "rsi_oversold", "above_sma20",
    "macd_positive", "volume_breakout", "volume_breakdown", "above_sma50",
    "ema7_above_ema21", "ema_cross_down", "rsi_overbought",
    "stoch_overbought", "stoch_oversold", "cci_overbought", "cci_oversold",
    "trending_market", "bullish_scenario_1", "bullish_scenario_2",
    "bullish_scenario_3", "bullish_scenario_4", "bullish_scenario_5",
    "bullish_scenario_6", "bearish_scenario_1", "bearish_scenario_2",
    "bearish_scenario_3", "bearish_scenario_4", "bearish_scenario_6",
    "close",
]
df.drop(columns=[c for c in DROP_COLS if c in df.columns], inplace=True)
df.dropna(inplace=True)
print(f"✅ Data: {len(df):,} rows, {df.shape[1]-1} features after cleaning")

X = df.drop(columns="target")
y = df["target"].astype(int)
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
cat_indices = [X.columns.get_loc(c) for c in cat_cols]
print(f"✅ Categorical columns: {len(cat_cols)}")

class_counts = np.bincount(y)
print(f"📊 Class balance 0/1: {class_counts}")

# ─── OPTUNA OBJECTIVE ───────────────────────────────────────

def objective(trial: optuna.Trial) -> float:
    # ----- balanced class weight upper bound (never <1) -------
    raw_pw = len(y) / max(1, 2 * class_counts[1])
    upper_pw = min(8.0, max(1.0, raw_pw))  # always ≥1

    # sample pos_weight only if useful
    if upper_pw > 1.0:
        pos_w = trial.suggest_float("pos_weight", 1.0, upper_pw)
        class_w = [1.0, pos_w]
    else:
        class_w = [1.0, 1.0]

    params = {
        "iterations": trial.suggest_int("iterations", 200, 1000),
        "learning_rate": trial.suggest_float("lr", 0.02, 0.15),
        "depth": trial.suggest_int("depth", 4, 8),
        "l2_leaf_reg": trial.suggest_float("l2", 2.0, 15.0),
        "random_strength": trial.suggest_float("rand_str", 1.0, 3.0),
        "min_data_in_leaf": trial.suggest_int("min_data_leaf", 10, 30),
        "class_weights": class_w,
        "bootstrap_type": trial.suggest_categorical("bootstrap", ["Bernoulli", "MVS", "Bayesian"]),
        "eval_metric": "F1",
        "loss_function": "Logloss",
        "random_seed": SEED,
        "task_type": "CPU",
        "thread_count": 4,
        "early_stopping_rounds": EARLY_STOP,
        "verbose": False,
    }

    # conditional hyper‑params --------------------------------
    if params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.6, 0.9)
    elif params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temp", 0.5, 3.0)
    # MVS does not need extra knobs and must not have "subsample"

    splitter = TimeSeriesSplit(n_splits=CV_FOLDS)
    scores = []

    for tr_idx, val_idx in splitter.split(X):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        if y_val.nunique() < 2:
            continue

        model = CatBoostClassifier(**params)
        model.fit(Pool(X_tr, y_tr, cat_features=cat_indices or None),
                  eval_set=Pool(X_val, y_val, cat_features=cat_indices or None))

        y_pred = (model.predict_proba(X_val)[:, 1] >= FIXED_THRESH).astype(int)
        scores.append(fbeta_score_fixed(y_val, y_pred))
        # ─── Optional metrics per trial fold ─────
        prec = precision_score(y_val, y_pred, zero_division=0)
        rec  = recall_score(y_val, y_pred, zero_division=0)
        f1   = f1_score(y_val, y_pred, zero_division=0)

        print(f"[Trial {trial.number}] Fold metrics – Precision: {prec:.3f} | Recall: {rec:.3f} | F1: {f1:.3f}")

        del model
        cleanup_memory()

    return float(np.mean(scores)) if scores else 0.0

# ─── RUN OPTIMISATION ───────────────────────────────────────
print("🚀 Hyper‑parameter search …")

study = optuna.create_study(direction="maximize",
                           sampler=optuna.samplers.TPESampler(seed=SEED))
study.optimize(objective, n_trials=N_TRIALS, timeout=None, show_progress_bar=True)

print("🏆 Best trial value:", study.best_value)

# ─── TRAIN FINAL MODEL ──────────────────────────────────────
print("🔧 Training final model …")

cut = int(len(df) * (1 - VAL_FRAC))
X_train, y_train = X.iloc[:cut], y.iloc[:cut]
X_test,  y_test  = X.iloc[cut:], y.iloc[cut:]

best_params = {
    "iterations": study.best_params.get("iterations", 500),
    "learning_rate": study.best_params.get("lr", 0.05),
    "depth": study.best_params.get("depth", 6),
    "l2_leaf_reg": study.best_params.get("l2", 5.0),
    "min_data_in_leaf": study.best_params.get("min_data_leaf", 15),
    "random_strength": study.best_params.get("rand_str", 1.5),
    "class_weights": [1.0, study.best_params.get("pos_weight", 1.0)],
    "bootstrap_type": study.best_params.get("bootstrap", "Bernoulli"),
    "eval_metric": "F1",
    "loss_function": "Logloss",
    "random_seed": SEED,
    "early_stopping_rounds": EARLY_STOP,
    "task_type": "CPU",
    "thread_count": 4,
    "verbose": 100,
}

if best_params["bootstrap_type"] == "Bernoulli":
    best_params["subsample"] = study.best_params.get("subsample", 0.8)
elif best_params["bootstrap_type"] == "Bayesian":
    best_params["bagging_temperature"] = study.best_params.get("bagging_temp", 1.0)

final_model = CatBoostClassifier(**best_params)
final_model.fit(Pool(X_train, y_train, cat_features=cat_indices or None),
                eval_set=Pool(X_test, y_test, cat_features=cat_indices or None))
print("✅ Final model trained")

# ─── EVALUATE ───────────────────────────────────────────────
print("📊 Evaluating …")

y_proba = final_model.predict_proba(X_test)[:, 1]
y_pred_fixed = (y_proba >= FIXED_THRESH).astype(int)

auc_fixed = roc_auc_score(y_test, y_proba)
f1_fixed  = f1_score(y_test, y_pred_fixed)
f05_fixed = fbeta_score_fixed(y_test, y_pred_fixed)

# Threshold sweep for best F0.5
thrs = np.linspace(0.1, 0.9, 17)
best_thr, best_f05 = FIXED_THRESH, f05_fixed
for thr in thrs:
    pred = (y_proba >= thr).astype(int)
    f05 = fbeta_score_fixed(y_test, pred)
    if f05 > best_f05:
        best_f05, best_thr = f05, thr

y_pred_opt = (y_proba >= best_thr).astype(int)

print("═"*60)
print(f"AUC‑ROC     : {auc_fixed:.4f}")
print(f"F1@0.5      : {f1_fixed:.4f}")
print(f"F0.5@0.5    : {f05_fixed:.4f}")
print(f"Best F0.5   : {best_f05:.4f}  (thr = {best_thr:.2f})")
print("═"*60)

# ─── SAVE ARTEFACTS ─────────────────────────────────────────
print("💾 Saving artefacts …")

out_dir = Path("catboost_f05_results")
out_dir.mkdir(exist_ok=True)

joblib.dump(study, out_dir / "optuna_study.pkl")
final_model.save_model(str(out_dir / "catboost_final.cbm"))

pd.DataFrame({
    "probability": y_proba,
    "pred_fixed": y_pred_fixed,
    "pred_optimal": y_pred_opt,
    "actual": y_test.values,
}).to_csv(out_dir / "predictions.csv")
]
print(f"✅ Files saved to {out_dir.resolve()}")


[I 2025-06-12 11:10:22,340] A new study created in memory with name: no-name-28b17c07-ed34-4a61-bac9-7eaf66c6d443


🖥️ Platform: Windows — running in CPU‑safe mode
📂 Loading data …
✅ Data: 15,855 rows, 31 features after cleaning
✅ Categorical columns: 0
📊 Class balance 0/1: [7758 8097]
🚀 Hyper‑parameter search …


  0%|          | 0/100 [00:00<?, ?it/s]

[Trial 0] Fold metrics – Precision: 0.545 | Recall: 0.581 | F1: 0.562
[Trial 0] Fold metrics – Precision: 0.514 | Recall: 0.718 | F1: 0.599


Best trial: 0. Best value: 0.551707:   1%|          | 1/100 [00:03<05:19,  3.23s/it]

...

Best trial: 98. Best value: 0.566487:  99%|█████████▉| 99/100 [03:36<00:01,  1.34s/it]

[Trial 98] Fold metrics – Precision: 0.565 | Recall: 0.596 | F1: 0.580
[I 2025-06-12 11:13:58,976] Trial 98 finished with value: 0.5664871124720557 and parameters: {'iterations': 752, 'lr': 0.020691023226819216, 'depth': 4, 'l2': 13.683470245201063, 'rand_str': 1.7369638634616382, 'min_data_leaf': 17, 'bootstrap': 'Bernoulli', 'subsample': 0.715299484815648}. Best is trial 98 with value: 0.5664871124720557.
[Trial 99] Fold metrics – Precision: 0.547 | Recall: 0.574 | F1: 0.561
[Trial 99] Fold metrics – Precision: 0.511 | Recall: 0.781 | F1: 0.618


Best trial: 98. Best value: 0.566487: 100%|██████████| 100/100 [03:39<00:00,  2.20s/it]

[Trial 99] Fold metrics – Precision: 0.545 | Recall: 0.601 | F1: 0.572
[I 2025-06-12 11:14:01,982] Trial 99 finished with value: 0.5522702962633894 and parameters: {'iterations': 737, 'lr': 0.021200075185411518, 'depth': 7, 'l2': 13.619680830900808, 'rand_str': 1.9388636570956166, 'min_data_leaf': 17, 'bootstrap': 'Bernoulli', 'subsample': 0.7127557257063355}. Best is trial 98 with value: 0.5664871124720557.
🏆 Best trial value: 0.5664871124720557
🔧 Training final model …
0:	learn: 0.5552593	test: 0.5676988	best: 0.5676988 (0)	total: 3.3ms	remaining: 2.48s


100:	learn: 0.5716047	test: 0.5072318	best: 0.5676988 (0)	total: 293ms	remaining: 1.89s
Stopped by overfitting detector  (150 iterations wait)

bestTest = 0.5676988464
bestIteration = 0

Shrink model to first 1 iterations.
✅ Final model trained
📊 Evaluating …
════════════════════════════════════════════════════════════
AUC‑ROC     : 0.5581
F1@0.5      : 0.5677
F0.5@0.5    : 0.5696
Best F0.5   : 0.5774  (thr = 0.10)
════════════════════════════════════════════════════════════
💾 Saving artefacts …
✅ Files saved to C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\catboost_f05_results


In [15]:
# =============================================================
#  MULTI-TRIAL CATBOOST EVALUATOR - Top 10 Configurations
#  Tests all top performing parameter sets on validation data
# =============================================================
import os, random, warnings, joblib, time
from pathlib import Path

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             fbeta_score, roc_auc_score, precision_score, 
                             recall_score, f1_score)

# ─── GLOBAL SEED ─────────────────────────────────────────────
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
random.seed(SEED)
warnings.filterwarnings("ignore")

# ─── CONFIG ─────────────────────────────────────────────────
CSV_PATH   = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
DROP_COLS  = [
    # Raw OHLC (massively correlated with close)
    'open', 'high', 'low',
    # ATR components
    'high_low', 'high_close', 'low_close',
    # Helper calculations  
    'typical_price', 'true_range', 'volume_mean_20',
    # Highly correlated MAs
    'EMA_21', 'SMA_20', 'vwap_24h', 'close_4h',
    # Bollinger components
    'bollinger_upper', 'bollinger_lower',
    # MACD components
    'MACD_line', 'MACD_signal',
    # Support/resistance
    'resistance_level', 'support_level',
    # Zero-importance binary flags
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    # Highly correlated position flags
    'above_sma50', 'ema7_above_ema21',
    # Low-importance cross signals
    'ema_cross_down',
    # Oscillator extremes
    'rsi_overbought', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold',
    # Trend flags
    'trending_market',
    # All scenario features
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'close'  # Target leakage prevention
]
VAL_FRAC   = 0.20
BETA       = 0.5  # F0.5 score (precision-weighted)
SAVE_DIR   = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models")

# ─── TOP 10 TRIAL CONFIGURATIONS FROM OPTIMIZATION RESULTS ──
TOP_TRIALS = [
    {
        "name": "Trial_98_Best_Overall",
        "trial": 98,
        "iterations": 752,
        "lr": 0.020691023226819216,
        "depth": 4,
        "l2": 13.683470245201063,
        "rand_str": 1.7369638634616382,
        "min_data_leaf": 17,
        "bootstrap": "Bernoulli",
        "subsample": 0.715299484815648,
        "cv_f05": 0.5665,
        "cv_precision": 0.565,
        "cv_recall": 0.596,
        "cv_f1": 0.580
    },
    {
        "name": "Trial_22_High_Precision", 
        "trial": 22,
        "iterations": 697,
        "lr": 0.05307592822927969,
        "depth": 4,
        "l2": 8.625798314956667,
        "rand_str": 1.9161178234306955,
        "min_data_leaf": 23,
        "bootstrap": "Bernoulli",
        "subsample": 0.7365237906217537,
        "cv_f05": 0.5655,
        "cv_precision": 0.562,
        "cv_recall": 0.590,
        "cv_f1": 0.576
    },
    {
        "name": "Trial_73_Balanced",
        "trial": 73,
        "iterations": 649,
        "lr": 0.024421356483576424,
        "depth": 4,
        "l2": 11.788492530962955,
        "rand_str": 1.8357264315461386,
        "min_data_leaf": 22,
        "bootstrap": "Bernoulli",
        "subsample": 0.7250562968464499,
        "cv_f05": 0.5649,
        "cv_precision": 0.558,
        "cv_recall": 0.592,
        "cv_f1": 0.574
    },
    {
        "name": "Trial_57_High_F1",
        "trial": 57,
        "iterations": 763,
        "lr": 0.04563393175332064,
        "depth": 4,
        "l2": 5.629181221838124,
        "rand_str": 1.2915802799684024,
        "min_data_leaf": 14,
        "bootstrap": "Bayesian",
        "bagging_temp": 2.2171824555502404,
        "cv_f05": 0.5642,
        "cv_precision": 0.552,
        "cv_recall": 0.634,
        "cv_f1": 0.590
    },
    {
        "name": "Trial_53_Top_Precision",
        "trial": 53,
        "iterations": 637,
        "lr": 0.035671612771177995,
        "depth": 4,
        "l2": 8.079433096935066,
        "rand_str": 1.7339338804777007,
        "min_data_leaf": 14,
        "bootstrap": "Bernoulli",
        "subsample": 0.7223730517623818,
        "cv_f05": 0.5638,
        "cv_precision": 0.565,
        "cv_recall": 0.594,
        "cv_f1": 0.579
    },
    {
        "name": "Trial_90_Max_Precision",
        "trial": 90,
        "iterations": 851,
        "lr": 0.05589556359158724,
        "depth": 4,
        "l2": 11.116885429251306,
        "rand_str": 2.4489985121801245,
        "min_data_leaf": 25,
        "bootstrap": "Bernoulli",
        "subsample": 0.8525508049455583,
        "cv_f05": 0.5626,
        "cv_precision": 0.566,
        "cv_recall": 0.552,
        "cv_f1": 0.559
    },
    {
        "name": "Trial_35_Conservative",
        "trial": 35,
        "iterations": 621,
        "lr": 0.033756991755374714,
        "depth": 4,
        "l2": 12.977478540926173,
        "rand_str": 2.058560862981183,
        "min_data_leaf": 20,
        "bootstrap": "Bernoulli",
        "subsample": 0.750613817369306,
        "cv_f05": 0.5629,
        "cv_precision": 0.562,
        "cv_recall": 0.587,
        "cv_f1": 0.574
    },
    {
        "name": "Trial_43_High_Recall",
        "trial": 43,
        "iterations": 683,
        "lr": 0.04304321773202563,
        "depth": 4,
        "l2": 12.555495688003719,
        "rand_str": 2.778037803783163,
        "min_data_leaf": 18,
        "bootstrap": "Bernoulli",
        "subsample": 0.7774776609307122,
        "cv_f05": 0.5629,
        "cv_precision": 0.536,
        "cv_recall": 0.697,
        "cv_f1": 0.606
    },
    {
        "name": "Trial_54_Stable",
        "trial": 54,
        "iterations": 632,
        "lr": 0.035144891267128345,
        "depth": 4,
        "l2": 6.489802684741546,
        "rand_str": 1.684970724115755,
        "min_data_leaf": 13,
        "bootstrap": "Bernoulli",
        "subsample": 0.7147121803375743,
        "cv_f05": 0.5631,
        "cv_precision": 0.557,
        "cv_recall": 0.571,
        "cv_f1": 0.564
    },
    {
        "name": "Trial_66_Alternative",
        "trial": 66,
        "iterations": 634,
        "lr": 0.03590178982335925,
        "depth": 4,
        "l2": 9.860110251607157,
        "rand_str": 1.9680368933611923,
        "min_data_leaf": 14,
        "bootstrap": "Bernoulli",
        "subsample": 0.7299161795768209,
        "cv_f05": 0.5628,
        "cv_precision": 0.562,
        "cv_recall": 0.589,
        "cv_f1": 0.575
    }
]

print("🚀 MULTI-TRIAL CATBOOST EVALUATION")
print(f"📊 Testing {len(TOP_TRIALS)} top configurations on validation data")
print("="*80)

# ─── DATA PREPARATION (IDENTICAL TO OPTIMIZATION) ───────────
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df[df.index >= "2018-01-01"]

if "target" not in df.columns:
    raise ValueError("❌ Missing 'target' column!")

categoricals = df.select_dtypes(include=["object", "category"]).columns.tolist()
categoricals = [c for c in categoricals if c != "target"]

df.drop(columns=[c for c in DROP_COLS if c in df.columns], inplace=True)
df.dropna(inplace=True)

X = df.drop(columns="target")
y = df["target"].astype(int)

cat_indices = [X.columns.get_loc(c) for c in categoricals if c in X.columns]

print(f"✅ Data shape: {X.shape}")
print(f"✅ Class distribution: {np.bincount(y)}")
print(f"✅ Categorical features: {len(cat_indices)}")

# ─── TRAIN/VALIDATION SPLIT ─────────────────────────────────
cut = int(len(df) * (1 - VAL_FRAC))
X_train, X_val = X.iloc[:cut], X.iloc[cut:]
y_train, y_val = y.iloc[:cut], y.iloc[cut:]

print(f"✅ Train size: {len(X_train):,}")
print(f"✅ Validation size: {len(X_val):,}")

# ─── EVALUATE EACH TRIAL CONFIGURATION ──────────────────────
results = []
best_model = None
best_score = 0
best_predictions = None

print(f"\n{'='*80}")
print("EVALUATING TOP TRIAL CONFIGURATIONS")
print(f"{'='*80}")

for i, trial_config in enumerate(TOP_TRIALS, 1):
    print(f"\n🔬 [{i}/{len(TOP_TRIALS)}] {trial_config['name']}")
    print(f"   Expected CV: P={trial_config['cv_precision']:.3f}, R={trial_config['cv_recall']:.3f}, F1={trial_config['cv_f1']:.3f}, F0.5={trial_config['cv_f05']:.3f}")
    
    start_time = time.time()
    
    try:
        # Build parameters for this trial
        params = {
            "iterations": trial_config["iterations"],
            "learning_rate": trial_config["lr"],
            "depth": trial_config["depth"],
            "l2_leaf_reg": trial_config["l2"],
            "random_strength": trial_config["rand_str"],
            "min_data_in_leaf": trial_config["min_data_leaf"],
            "bootstrap_type": trial_config["bootstrap"],
            "random_seed": SEED,
            "task_type": "GPU",
            "verbose": False,
            "use_best_model": True,
            "loss_function": "Logloss",
            "eval_metric": "F1",
            "early_stopping_rounds": min(100, trial_config["iterations"] // 4)
        }
        
        # Add bootstrap-specific parameters
        if trial_config["bootstrap"] == "Bernoulli":
            params["subsample"] = trial_config["subsample"]
        elif trial_config["bootstrap"] == "Bayesian":
            params["bagging_temperature"] = trial_config["bagging_temp"]
        
        # Train model
        model = CatBoostClassifier(**params)
        
        train_pool = Pool(X_train, y_train, cat_features=cat_indices if cat_indices else None)
        val_pool = Pool(X_val, y_val, cat_features=cat_indices if cat_indices else None)
        
        model.fit(train_pool, eval_set=val_pool)
        
        # Get predictions
        y_prob = model.predict_proba(X_val)[:, 1]
        y_pred_50 = (y_prob >= 0.5).astype(int)
        
        # Calculate metrics at 0.5 threshold
        precision_50 = precision_score(y_val, y_pred_50, zero_division=0)
        recall_50 = recall_score(y_val, y_pred_50, zero_division=0)
        f1_50 = f1_score(y_val, y_pred_50, zero_division=0)
        f05_50 = fbeta_score(y_val, y_pred_50, beta=BETA, zero_division=0)
        
        # Find optimal threshold for F0.5
        thresholds = np.arange(0.1, 0.9, 0.05)
        best_thresh = 0.5
        best_f05 = f05_50
        
        for thresh in thresholds:
            y_pred_thresh = (y_prob >= thresh).astype(int)
            f05_thresh = fbeta_score(y_val, y_pred_thresh, beta=BETA, zero_division=0)
            if f05_thresh > best_f05:
                best_f05 = f05_thresh
                best_thresh = thresh
        
        # Calculate metrics at optimal threshold
        y_pred_opt = (y_prob >= best_thresh).astype(int)
        precision_opt = precision_score(y_val, y_pred_opt, zero_division=0)
        recall_opt = recall_score(y_val, y_pred_opt, zero_division=0)
        f1_opt = f1_score(y_val, y_pred_opt, zero_division=0)
        f05_opt = fbeta_score(y_val, y_pred_opt, beta=BETA, zero_division=0)
        
        accuracy = accuracy_score(y_val, y_pred_opt)
        auc = roc_auc_score(y_val, y_prob)
        
        train_time = time.time() - start_time
        
        # Store results
        result = {
            "name": trial_config["name"],
            "trial": trial_config["trial"],
            "train_time": train_time,
            # CV metrics for comparison
            "cv_precision": trial_config["cv_precision"],
            "cv_recall": trial_config["cv_recall"],
            "cv_f1": trial_config["cv_f1"],
            "cv_f05": trial_config["cv_f05"],
            # Validation metrics at 0.5 threshold
            "val_precision_50": precision_50,
            "val_recall_50": recall_50,
            "val_f1_50": f1_50,
            "val_f05_50": f05_50,
            # Validation metrics at optimal threshold
            "optimal_threshold": best_thresh,
            "val_precision_opt": precision_opt,
            "val_recall_opt": recall_opt,
            "val_f1_opt": f1_opt,
            "val_f05_opt": f05_opt,
            "val_accuracy": accuracy,
            "val_auc": auc,
            "predictions": y_prob,
            "predictions_binary": y_pred_opt
        }
        
        results.append(result)
        
        print(f"   ⏱️ Training time: {train_time:.1f}s")
        print(f"   📊 Val @ 0.5   : P={precision_50:.3f}, R={recall_50:.3f}, F1={f1_50:.3f}, F0.5={f05_50:.3f}")
        print(f"   🎯 Val @ {best_thresh:.2f}  : P={precision_opt:.3f}, R={recall_opt:.3f}, F1={f1_opt:.3f}, F0.5={f05_opt:.3f}")
        print(f"   ✅ AUC: {auc:.3f}, Accuracy: {accuracy:.3f}")
        
        # Track best model
        if f05_opt > best_score:
            best_score = f05_opt
            best_model = result
            best_predictions = y_prob
            
    except Exception as e:
        print(f"   ❌ Training failed: {str(e)[:50]}...")
        continue

# ─── RESULTS SUMMARY ────────────────────────────────────────
print(f"\n{'='*80}")
print("FINAL RESULTS SUMMARY")
print(f"{'='*80}")

if results:
    # Sort by F0.5 score
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('val_f05_opt', ascending=False)
    
    print(f"\n🏆 RANKING BY VALIDATION F0.5 SCORE:")
    print("-" * 120)
    print(f"{'Rank':<4} {'Trial':<25} {'Val F0.5':<8} {'Val Prec':<9} {'Val Rec':<8} {'Val F1':<7} {'Thresh':<7} {'CV F0.5':<7} {'Δ F0.5':<7}")
    print("-" * 120)
    
    for rank, (_, row) in enumerate(results_df.iterrows(), 1):
        delta_f05 = row['val_f05_opt'] - row['cv_f05']
        print(f"{rank:<4} {row['name']:<25} {row['val_f05_opt']:<8.3f} {row['val_precision_opt']:<9.3f} {row['val_recall_opt']:<8.3f} {row['val_f1_opt']:<7.3f} {row['optimal_threshold']:<7.2f} {row['cv_f05']:<7.3f} {delta_f05:+.3f}")
    
    print("-" * 120)
    
    # Best model details
    best_trial = results_df.iloc[0]
    print(f"\n🥇 BEST MODEL: {best_trial['name']}")
    print(f"   📈 Validation F0.5: {best_trial['val_f05_opt']:.4f}")
    print(f"   🎯 Optimal threshold: {best_trial['optimal_threshold']:.3f}")
    print(f"   📊 Precision: {best_trial['val_precision_opt']:.3f}")
    print(f"   📊 Recall: {best_trial['val_recall_opt']:.3f}")
    print(f"   📊 F1 Score: {best_trial['val_f1_opt']:.3f}")
    print(f"   📊 Accuracy: {best_trial['val_accuracy']:.3f}")
    print(f"   📊 AUC-ROC: {best_trial['val_auc']:.3f}")
    
    # Performance vs CV comparison
    print(f"\n📊 CV vs VALIDATION COMPARISON:")
    avg_cv_f05 = results_df['cv_f05'].mean()
    avg_val_f05 = results_df['val_f05_opt'].mean()
    print(f"   Average CV F0.5: {avg_cv_f05:.3f}")
    print(f"   Average Val F0.5: {avg_val_f05:.3f}")
    print(f"   Overall difference: {avg_val_f05 - avg_cv_f05:+.3f}")
    
    # Save best predictions
    best_pred_df = pd.DataFrame({
        "prob_up": best_trial['predictions'],
        f"pred_{best_trial['optimal_threshold']:.2f}": best_trial['predictions_binary'],
        "actual": y_val.values
    }, index=X_val.index)
    
    save_path = SAVE_DIR / "catboost_best_trial_predictions.csv"
    best_pred_df[["prob_up", f"pred_{best_trial['optimal_threshold']:.2f}"]].to_csv(save_path)
    print(f"\n💾 Best model predictions saved: {save_path}")
    
    # Save all results
    results_summary = {
        "best_trial": best_trial['name'],
        "best_f05": best_trial['val_f05_opt'],
        "all_results": results_df.to_dict('records'),
        "summary_stats": {
            "avg_cv_f05": avg_cv_f05,
            "avg_val_f05": avg_val_f05,
            "best_improvement": results_df['val_f05_opt'].max() - results_df['cv_f05'].max(),
            "trials_completed": len(results)
        }
    }
    
    joblib.dump(results_summary, SAVE_DIR / "multi_trial_results.pkl")
    print(f"📁 Detailed results saved: {SAVE_DIR / 'multi_trial_results.pkl'}")
    
else:
    print("❌ No trials completed successfully!")

print(f"\n🎉 EVALUATION COMPLETED!")
print(f"✅ {len(results)} out of {len(TOP_TRIALS)} trials completed successfully")

🚀 MULTI-TRIAL CATBOOST EVALUATION
📊 Testing 10 top configurations on validation data
✅ Data shape: (15855, 31)
✅ Class distribution: [7758 8097]
✅ Categorical features: 0
✅ Train size: 12,684
✅ Validation size: 3,171

EVALUATING TOP TRIAL CONFIGURATIONS

🔬 [1/10] Trial_98_Best_Overall
   Expected CV: P=0.565, R=0.596, F1=0.580, F0.5=0.567
   ⏱️ Training time: 2.0s
   📊 Val @ 0.5   : P=0.558, R=0.588, F1=0.572, F0.5=0.563
   🎯 Val @ 0.10  : P=0.522, R=1.000, F1=0.686, F0.5=0.577
   ✅ AUC: 0.549, Accuracy: 0.522

🔬 [2/10] Trial_22_High_Precision
   Expected CV: P=0.562, R=0.590, F1=0.576, F0.5=0.566
   ⏱️ Training time: 1.9s
   📊 Val @ 0.5   : P=0.546, R=0.612, F1=0.577, F0.5=0.558
   🎯 Val @ 0.10  : P=0.522, R=1.000, F1=0.686, F0.5=0.577
   ✅ AUC: 0.545, Accuracy: 0.522

🔬 [3/10] Trial_73_Balanced
   Expected CV: P=0.558, R=0.592, F1=0.574, F0.5=0.565
   ⏱️ Training time: 1.8s
   📊 Val @ 0.5   : P=0.546, R=0.612, F1=0.577, F0.5=0.558
   🎯 Val @ 0.10  : P=0.522, R=1.000, F1=0.686, F0.5=0

In [16]:
print(f"{'Rank':<4} {'Trial':<25} {'F0.5@0.5':<8} {'Prec@0.5':<9} {'Rec@0.5':<8} {'F1@0.5':<7} {'CV F0.5':<7} {'Δ F0.5':<7}")
...
for rank, (_, row) in enumerate(results_df.iterrows(), 1):
    delta_f05 = row['val_f05_50'] - row['cv_f05']
    print(f"{rank:<4} {row['name']:<25} {row['val_f05_50']:<8.3f} {row['val_precision_50']:<9.3f} {row['val_recall_50']:<8.3f} {row['val_f1_50']:<7.3f} {row['cv_f05']:<7.3f} {delta_f05:+.3f}")


Rank Trial                     F0.5@0.5 Prec@0.5  Rec@0.5  F1@0.5  CV F0.5 Δ F0.5 
1    Trial_98_Best_Overall     0.563    0.558     0.588    0.572   0.567   -0.003
2    Trial_22_High_Precision   0.558    0.546     0.612    0.577   0.566   -0.008
3    Trial_73_Balanced         0.558    0.546     0.612    0.577   0.565   -0.007
4    Trial_57_High_F1          0.557    0.575     0.495    0.532   0.564   -0.008
5    Trial_53_Top_Precision    0.558    0.546     0.612    0.577   0.564   -0.006
6    Trial_90_Max_Precision    0.558    0.546     0.612    0.577   0.563   -0.005
7    Trial_35_Conservative     0.558    0.546     0.612    0.577   0.563   -0.005
8    Trial_43_High_Recall      0.558    0.546     0.612    0.577   0.563   -0.005
9    Trial_54_Stable           0.558    0.546     0.612    0.577   0.563   -0.006
10   Trial_66_Alternative      0.558    0.546     0.612    0.577   0.563   -0.005


In [2]:
# =============================================================
#  BEST PRECISION MODEL TRAINER & PREDICTION CSV GENERATOR
# =============================================================
import numpy as np
import pandas as pd
import time
import sys
import warnings
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, 
    f1_score, roc_auc_score, classification_report, confusion_matrix
)
from sklearn.feature_selection import VarianceThreshold
import joblib

warnings.filterwarnings("ignore")
np.random.seed(42)

# ──────────────────────────────────────────────────────────────
# 1) CONFIGURATION
# ──────────────────────────────────────────────────────────────
CSV_FILE = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
TIME_COL = "timestamp"
TARGET_COL = "target"
START_DATE = "2018-01-01"
TEST_FRAC = 0.20
RANDOM_STATE = 42

# Output paths
OUTPUT_DIR = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\Predictions_folder")
OUTPUT_DIR.mkdir(exist_ok=True)
PREDICTIONS_CSV = OUTPUT_DIR / "validation_predictions.csv"
MODEL_PATH = OUTPUT_DIR / "best_precision_model.pkl"

# BEST PRECISION PARAMETERS (Original_Best_ElasticNet)
BEST_PRECISION_PARAMS = {
    'C': 0.0016351310838425184,
    'class_weight': None,
    'l1_ratio': 0.2636043819680166,
    'penalty': 'elasticnet',
    'solver': 'saga',
    'max_iter': 5000,
    'random_state': RANDOM_STATE
}

DROP_COLS = [
    'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'overbought_reversal', 'oversold_reversal', 'macd_cross_up',
    'macd_cross_down', 'macd_rising', 'bollinger_upper', 'bollinger_lower',
    'MACD_line', 'MACD_signal', 'stoch_%D', 'momentum_alignment',
    'bullish_scenario_1', 'bullish_scenario_5', 'bearish_scenario_1'
]

# ──────────────────────────────────────────────────────────────
# 2) DATA LOADING & PREPROCESSING
# ──────────────────────────────────────────────────────────────
def load_and_prepare_data():
    """Load and prepare data for best precision model training."""
    print("🏆 BEST PRECISION MODEL TRAINING")
    print("=" * 50)
    
    if not CSV_FILE.exists():
        sys.exit(f"❌ File not found: {CSV_FILE}")
    
    print(f"📂 Loading data from: {CSV_FILE}")
    df = pd.read_csv(CSV_FILE, parse_dates=[TIME_COL]).set_index(TIME_COL).sort_index()
    df = df.loc[START_DATE:].copy()
    
    if TARGET_COL not in df.columns:
        sys.exit(f"❌ '{TARGET_COL}' column missing!")
    
    # Remove specified columns
    X = df.drop(columns=[c for c in DROP_COLS if c in df.columns] + [TARGET_COL], errors="ignore")
    y = df[TARGET_COL]
    
    print(f"📊 Dataset shape: {X.shape}")
    print(f"📈 Date range: {df.index.min()} to {df.index.max()}")
    print(f"🎯 Target distribution: {y.value_counts().to_dict()}")
    
    # Handle missing and infinite values
    original_size = len(X)
    
    # Check for missing values
    missing_vals = X.isnull().sum()
    if missing_vals.sum() > 0:
        print(f"⚠️ Handling {missing_vals.sum()} missing values...")
        mask = ~(X.isnull().any(axis=1) | y.isnull())
        X, y = X[mask], y[mask]
        print(f"📊 Shape after removing missing: {X.shape}")
    
    # Check for infinite values
    inf_mask = np.isinf(X.select_dtypes(include=[np.number])).any(axis=1)
    if inf_mask.sum() > 0:
        print(f"⚠️ Handling {inf_mask.sum()} infinite values...")
        X, y = X[~inf_mask], y[~inf_mask]
        print(f"📊 Final shape: {X.shape}")
    
    # Keep track of valid indices for timestamps
    if missing_vals.sum() > 0 or inf_mask.sum() > 0:
        if missing_vals.sum() > 0:
            valid_mask = ~(df.drop(columns=[c for c in DROP_COLS if c in df.columns] + [TARGET_COL], errors="ignore").isnull().any(axis=1) | df[TARGET_COL].isnull())
        else:
            valid_mask = slice(None)
        
        if inf_mask.sum() > 0:
            if isinstance(valid_mask, slice):
                valid_mask = ~inf_mask
            else:
                valid_mask = valid_mask & ~inf_mask
        
        timestamps = df.index[valid_mask]
    else:
        timestamps = df.index
    
    removed_samples = original_size - len(X)
    if removed_samples > 0:
        print(f"📉 Removed {removed_samples} samples due to missing/infinite values")
    
    return X, y, timestamps

def preprocess_features(X_train, X_test):
    """Preprocess features with variance filtering."""
    print("\n🔧 Preprocessing features...")
    
    # Remove constant features
    constant_cols = X_train.columns[X_train.std() == 0]
    if len(constant_cols) > 0:
        print(f"⚠️ Removing {len(constant_cols)} constant features")
        X_train = X_train.drop(columns=constant_cols)
        X_test = X_test.drop(columns=constant_cols)
    
    # Remove low-variance features
    variance_selector = VarianceThreshold(threshold=0.01)
    n_features_before = X_train.shape[1]
    
    X_train_selected = pd.DataFrame(
        variance_selector.fit_transform(X_train),
        index=X_train.index,
        columns=X_train.columns[variance_selector.get_support()]
    )
    X_test_selected = pd.DataFrame(
        variance_selector.transform(X_test),
        index=X_test.index,
        columns=X_train.columns[variance_selector.get_support()]
    )
    
    n_features_after = X_train_selected.shape[1]
    if n_features_before != n_features_after:
        print(f"⚠️ Removed {n_features_before - n_features_after} low-variance features")
    
    print(f"✅ Final feature count: {n_features_after}")
    return X_train_selected, X_test_selected, variance_selector

# ──────────────────────────────────────────────────────────────
# 3) MODEL TRAINING
# ──────────────────────────────────────────────────────────────
def create_best_precision_pipeline():
    """Create pipeline with best precision parameters."""
    return Pipeline([
        ('scaler', StandardScaler()),
        ('logreg', LogisticRegression(**BEST_PRECISION_PARAMS))
    ])

def train_best_precision_model(X_train, y_train):
    """Train the best precision model."""
    print("\n🎯 TRAINING BEST PRECISION MODEL")
    print("=" * 45)
    
    pipeline = create_best_precision_pipeline()
    
    print("🏆 Using best precision parameters:")
    for param, value in BEST_PRECISION_PARAMS.items():
        print(f"   {param:<15}: {value}")
    
    print("\n⏱️ Training model...")
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    print(f"✅ Model trained in {training_time:.2f} seconds")
    
    # Check convergence
    logreg = pipeline.named_steps['logreg']
    if hasattr(logreg, 'n_iter_'):
        n_iter = logreg.n_iter_[0] if isinstance(logreg.n_iter_, np.ndarray) else logreg.n_iter_
        if n_iter >= logreg.max_iter:
            print(f"⚠️ Model may not have converged (used {n_iter}/{logreg.max_iter} iterations)")
        else:
            print(f"✅ Model converged in {n_iter} iterations")
    
    return pipeline, training_time

# ──────────────────────────────────────────────────────────────
# 4) MODEL EVALUATION
# ──────────────────────────────────────────────────────────────
def f_beta_half(y_true, y_pred):
    """Custom F-beta score with beta=0.5 (precision-weighted)."""
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    beta = 0.5
    if (p + r) == 0:
        return 0.0
    return (1 + beta**2) * p * r / (beta**2 * p + r)

def evaluate_best_precision_model(model, X_train, y_train, X_test, y_test):
    """Evaluate the best precision model."""
    print("\n📊 MODEL EVALUATION")
    print("=" * 30)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_train_prob = model.predict_proba(X_train)
    
    y_test_pred = model.predict(X_test)
    y_test_prob = model.predict_proba(X_test)
    
    # Calculate metrics for test set
    test_metrics = {
        'accuracy': accuracy_score(y_test, y_test_pred),
        'precision': precision_score(y_test, y_test_pred, zero_division=0),
        'recall': recall_score(y_test, y_test_pred, zero_division=0),
        'f1': f1_score(y_test, y_test_pred, zero_division=0),
        'f_beta_0.5': f_beta_half(y_test, y_test_pred),
        'roc_auc': roc_auc_score(y_test, y_test_prob[:, 1]) if len(np.unique(y_test)) > 1 else 0.0
    }
    
    print("📈 TEST SET METRICS:")
    print("-" * 25)
    for metric_name, value in test_metrics.items():
        print(f"   {metric_name:<12}: {value:.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_test_pred)
    print(f"\n🎯 CONFUSION MATRIX:")
    print("-" * 25)
    print(f"True Negatives:  {cm[0,0]:>6}")
    print(f"False Positives: {cm[0,1]:>6}")
    print(f"False Negatives: {cm[1,0]:>6}")
    print(f"True Positives:  {cm[1,1]:>6}")
    
    return {
        'test_metrics': test_metrics,
        'train_predictions': y_train_pred,
        'train_probabilities': y_train_prob,
        'test_predictions': y_test_pred,
        'test_probabilities': y_test_prob
    }

# ──────────────────────────────────────────────────────────────
# 5) PREDICTION CSV GENERATION
# ──────────────────────────────────────────────────────────────
def generate_predictions_csv(X_test, y_test, results, test_timestamps):
    """Generate CSV with ONLY validation (test) predictions in exact requested format."""
    print("\n💾 GENERATING VALIDATION PREDICTIONS CSV")
    print("=" * 45)
    
    # Get test predictions and probabilities
    test_predictions = results['test_predictions']
    test_probabilities = results['test_probabilities']
    
    # Create DataFrame in exact requested format
    predictions_df = pd.DataFrame({
        'timestamp': test_timestamps,
        'prob_up': test_probabilities[:, 1],      # Probability of class 1 (up)
        'prob_down': test_probabilities[:, 0],    # Probability of class 0 (down)
        'winning_prob': np.maximum(test_probabilities[:, 1], test_probabilities[:, 0]),  # Maximum probability
        'prediction': test_predictions,           # Final prediction (0 or 1)
        'actual': y_test.values                   # Actual target value (0 or 1)
    })
    
    # Sort by timestamp
    predictions_df = predictions_df.sort_values('timestamp').reset_index(drop=True)
    
    # Format timestamp to match your example (DD/MM/YYYY HH:MM)
    predictions_df['timestamp'] = predictions_df['timestamp'].dt.strftime('%d/%m/%Y %H:%M')
    
    # Save to CSV
    predictions_df.to_csv(PREDICTIONS_CSV, index=False)
    
    print(f"✅ Validation predictions saved to: {PREDICTIONS_CSV}")
    print(f"📊 Total validation predictions: {len(predictions_df):,}")
    
    # Display sample predictions in your exact format
    print(f"\n📋 SAMPLE PREDICTIONS (First 10 rows in your format):")
    print("-" * 80)
    print("timestamp            prob_up  prob_down winning_prob prediction actual")
    print("-" * 80)
    for idx, row in predictions_df.head(10).iterrows():
        print(f"{row['timestamp']:<20} {row['prob_up']:.6f} {row['prob_down']:.6f} {row['winning_prob']:.6f} {row['prediction']:<10} {row['actual']}")
    
    # Calculate validation accuracy
    accuracy = (predictions_df['prediction'] == predictions_df['actual']).mean()
    
    # Summary statistics
    print(f"\n📈 VALIDATION SUMMARY:")
    print("-" * 25)
    print(f"Validation accuracy: {accuracy:.4f}")
    print(f"Average winning probability: {predictions_df['winning_prob'].mean():.4f}")
    print(f"High confidence predictions (>0.7): {(predictions_df['winning_prob'] > 0.7).sum():,}")
    print(f"Correct high confidence predictions: {((predictions_df['winning_prob'] > 0.7) & (predictions_df['prediction'] == predictions_df['actual'])).sum():,}")
    
    return predictions_df

# ──────────────────────────────────────────────────────────────
# 6) MODEL SAVING
# ──────────────────────────────────────────────────────────────
def save_best_precision_model(model, variance_selector, feature_names, results, training_time):
    """Save the complete best precision model."""
    print(f"\n💾 SAVING BEST PRECISION MODEL")
    print("=" * 35)
    
    model_package = {
        'model': model,
        'variance_selector': variance_selector,
        'feature_names': feature_names,
        'parameters': BEST_PRECISION_PARAMS,
        'performance_metrics': results['test_metrics'],
        'training_info': {
            'training_date': pd.Timestamp.now(),
            'training_time_seconds': training_time,
            'algorithm': 'LogisticRegression ElasticNet (Best Precision)',
            'total_features': len(feature_names),
            'preprocessing_steps': ['StandardScaler', 'VarianceThreshold'],
            'model_description': 'Highest precision model from parameter comparison'
        }
    }
    
    joblib.dump(model_package, MODEL_PATH)
    print(f"✅ Model saved to: {MODEL_PATH}")
    
    return model_package

# ──────────────────────────────────────────────────────────────
# 7) MAIN EXECUTION
# ──────────────────────────────────────────────────────────────
def main():
    """Main execution function."""
    
    # Load and prepare data
    X, y, timestamps = load_and_prepare_data()
    
    # Chronological split
    split_idx = int(len(X) * (1 - TEST_FRAC))
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
    train_timestamps = timestamps[:split_idx]
    test_timestamps = timestamps[split_idx:]
    
    print(f"\n📊 TRAIN/TEST SPLIT:")
    print("-" * 25)
    print(f"   Train: {X_train.shape[0]:,} samples")
    print(f"   Test:  {X_test.shape[0]:,} samples")
    print(f"   Train period: {train_timestamps.min()} to {train_timestamps.max()}")
    print(f"   Test period:  {test_timestamps.min()} to {test_timestamps.max()}")
    
    # Preprocess features
    X_train_processed, X_test_processed, variance_selector = preprocess_features(X_train, X_test)
    
    # Train best precision model
    best_model, training_time = train_best_precision_model(X_train_processed, y_train)
    
    # Evaluate model
    results = evaluate_best_precision_model(best_model, X_train_processed, y_train, X_test_processed, y_test)
    
    # Generate validation predictions CSV (test set only)
    predictions_df = generate_predictions_csv(
        X_test_processed, y_test, results, test_timestamps
    )
    
    # Save model
    model_package = save_best_precision_model(
        best_model, variance_selector, list(X_train_processed.columns), 
        results, training_time
    )
    
    print(f"\n🎉 BEST PRECISION MODEL TRAINING COMPLETED!")
    print("=" * 50)
    print(f"📁 Files Generated:")
    print(f"   🔸 Predictions: {PREDICTIONS_CSV.name}")
    print(f"   🔸 Model:       {MODEL_PATH.name}")
    print(f"\n🏆 Final Test Precision: {results['test_metrics']['precision']:.4f}")
    print(f"🎯 Final Test F1-Score:  {results['test_metrics']['f1']:.4f}")
    
    return {
        'model': best_model,
        'predictions_df': predictions_df,
        'results': results,
        'model_package': model_package
    }

# ──────────────────────────────────────────────────────────────
# 8) SCRIPT EXECUTION
# ──────────────────────────────────────────────────────────────
if __name__ == "__main__":
    final_results = main()

🏆 BEST PRECISION MODEL TRAINING
📂 Loading data from: C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv
📊 Dataset shape: (15855, 46)
📈 Date range: 2018-01-01 00:00:00 to 2025-03-28 00:00:00
🎯 Target distribution: {1: 8097, 0: 7758}

📊 TRAIN/TEST SPLIT:
-------------------------
   Train: 12,684 samples
   Test:  3,171 samples
   Train period: 2018-01-01 00:00:00 to 2023-10-16 12:00:00
   Test period:  2023-10-16 16:00:00 to 2025-03-28 00:00:00

🔧 Preprocessing features...
⚠️ Removed 2 low-variance features
✅ Final feature count: 44

🎯 TRAINING BEST PRECISION MODEL
🏆 Using best precision parameters:
   C              : 0.0016351310838425184
   class_weight   : None
   l1_ratio       : 0.2636043819680166
   penalty        : elasticnet
   solver         : saga
   max_iter       : 5000
   random_state   : 42

⏱️ Training model...
✅ Model trained in 0.10 seconds
✅ Model converged in 15 iterations

📊 MODEL EVA

In [24]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the predictions CSV
csv_path = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\catboost_trial57_predictions.csv"
df = pd.read_csv(csv_path)

# Ensure column names are correct and lowercase
df.columns = df.columns.str.strip().str.lower()

# Extract actual and predicted values
y_true = df['actual']
y_pred = df['prediction']  # prediction at threshold 0.5

# Calculate metrics
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Print results
print("📊 Evaluation at threshold 0.5:")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1 Score : {f1:.3f}")


📊 Evaluation at threshold 0.5:
Precision: 0.575
Recall   : 0.495
F1 Score : 0.532
